In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import pandas as pd
pd.set_option('display.max_columns', None)

<center>
    <img src="https://logodownload.org/wp-content/uploads/2016/03/premier-league-5.png" width="75" alt="cognitiveclass.ai logo">
</center>

# Premier League API Retrieval, Data Collection and Visualization: Create & Visualize FPL Performance Metrics

<!-- Estimated time needed: **15** minutes -->

## Functions of notebook

This notebook was built to:

*   Retrieve data from Premier League API
-   Create attributes with fetched data to effectively collect and compile data.
*   Query data from attributres/functions to build visualization and statistics


In [ ]:
import sys
import os
sys.path.append(os.path.abspath('..'))
from src.functions.data_grabber import DataGrabberFPL
from src.functions.data_analysis import DataAnalytics
from src.functions.data_visualization import VisualizationOperations

fpl_data_attributes = DataGrabberFPL()
data_analytics = DataAnalytics(fpl_data_attributes)

In [ ]:
data_visualizer = VisualizationOperations(fpl_data_attributes, data_analytics)

---

### League Analysis

In [ ]:
league_rank_data = fpl_data_attributes.helper_fns.get_rank_data(782655)

In [ ]:
# league_rank_data

---

## Performance Spread

In [ ]:
full_sum = data_visualizer.display_tabular_summary()

In [ ]:
fpl_sum = data_visualizer.build_player_tabular_summary(list(data_analytics.personal_team_data.keys()))

In [ ]:
names = ['solanke', 'mateta', 'eze', 'henderson', 'saka', 'havertz']

values = [fpl_data_attributes.helper_fns.find_best_match(name) for name in names]
values += list(data_analytics.personal_team_data.keys())
custom_sum = data_visualizer.build_player_tabular_summary(values)

---

## Exploratory Data Analysis

In [ ]:
from copy import deepcopy
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
latest_data_from_fpl = deepcopy(fpl_data_attributes.master_summary)

### Box Plot Visualization

In [ ]:
def generate_zipped_params(param_list, reference_dict):
    parsed_data = [{param: data[param] for param in param_list} for player_id, data in reference_dict.items()]
    result = []
    keys = list(parsed_data[0].keys())
    for d in parsed_data:
        for i in range(len(d[keys[0]])):
            if all(d[key][i][0] == d[keys[0]][i][0] for key in d.keys()):
                result.append(tuple(d[key][i][1] for key in d.keys()))
    return result

def generate_box_plot_for_params(y_param, x_param, data):
    
    matched_tuples = generate_zipped_params([y_param,x_param], data)
    
    reference_df = pd.DataFrame(matched_tuples, columns=[y_param, x_param])
    reference_df = reference_df.loc[reference_df[x_param] > 0]
    
    grouped = reference_df.groupby(x_param)[y_param].apply(list)

    # Plotting
    plt.figure(figsize=(20, 6))
    plt.boxplot(grouped.values)
    plt.xticks(range(1, len(grouped) + 1), grouped.index)
    plt.xlabel(x_param)
    plt.ylabel(y_param)
    plt.title(f"Box plots for every '{x_param}' value against attained '{y_param}'")

    # Add transparent rectangles
    if x_param == "total_points":
        plt.gca().add_patch(patches.Rectangle((0.5, plt.ylim()[0]), 3, plt.ylim()[1]-plt.ylim()[0],
                                              alpha=0.3, color='red'))
        plt.gca().add_patch(patches.Rectangle((3.5, plt.ylim()[0]), 2, plt.ylim()[1]-plt.ylim()[0],
                                              alpha=0.3, color='yellow'))
        plt.gca().add_patch(patches.Rectangle((5.5, plt.ylim()[0]), 4, plt.ylim()[1]-plt.ylim()[0],
                                              alpha=0.3, color='green'))
        plt.gca().add_patch(patches.Rectangle((9.5, plt.ylim()[0]), 22, plt.ylim()[1]-plt.ylim()[0],
                                             alpha=0.3, color='blue'))
    plt.show()

In [ ]:
latest_data_from_fpl[1].keys()

In [ ]:
generate_box_plot_for_params(
    y_param = 'bps', 
    x_param = 'total_points',
    data = latest_data_from_fpl
)

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Generate synthetic impedance data with two relaxation processes
np.random.seed(0)
frequencies = np.logspace(0, 4, 100)  # Frequency range
Z_real = 10 + np.random.normal(0, 1, 100)  # Real part of impedance
Z_imag = 5 + np.random.normal(0, 1, 100)  # Imaginary part of impedance
Z = Z_real + 1j * Z_imag  # Complex impedance

# Define the Laplace distribution function
def laplace_dist(t, b, c):
    return (1/(2*b)) * np.exp(-np.abs(t-c)/b)

# Fit the Laplace distribution to the impedance data
popt, pcov = curve_fit(laplace_dist, frequencies, np.abs(Z), p0=[1, 1])

# Extract the parameters of the Laplace distribution
b, c = popt

# Plot the fitted distribution
plt.figure()
plt.scatter(frequencies, np.abs(Z), label='Impedance data')
plt.plot(frequencies, laplace_dist(frequencies, b, c), label='Fitted Laplace distribution')
plt.xscale('log')
plt.xlabel('Frequency')
plt.ylabel('Impedance Magnitude')
plt.legend()
plt.show()

# Print the extracted parameters
print(f'Scale parameter (b): {b}')
print(f'Location parameter (c): {c}')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Generate example input and output signals
# Let's assume the signals are arrays of sinusoidal waves with some noise
time = np.linspace(0, 1, 1000)
frequency = 10  # Frequency of the sinusoidal wave
amplitude = 1  # Amplitude of the sinusoidal wave
input_signal = amplitude * np.sin(2 * np.pi * frequency * time) + np.random.normal(0, 0.1, len(time))
output_signal = amplitude * np.sin(2 * np.pi * frequency * time + np.pi/4) + np.random.normal(0, 0.1, len(time))

# Calculate FFT of input and output signals
fft_input = np.fft.fft(input_signal)
fft_output = np.fft.fft(output_signal)

# Calculate complex impedance
complex_impedance = fft_output / fft_input

# Calculate magnitude of impedance
magnitude_impedance = np.abs(complex_impedance)

# Plot the magnitude of impedance (Bode plot)
frequency_axis = np.fft.fftfreq(len(time), time[1] - time[0])
plt.figure()
plt.semilogx(frequency_axis[:len(frequency_axis)//2], magnitude_impedance[:len(frequency_axis)//2])
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude of Impedance')
plt.title('Bode Plot')
plt.grid(True)
plt.show()
